## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import time

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Weather Description
0,0,Tiksi,71.6872,128.8694,-25.56,86,100,5.82,RU,2021-02-06 18:05:26,overcast clouds
1,1,Sinnamary,5.3833,-52.9500,79.52,85,91,16.06,GF,2021-02-06 18:05:26,overcast clouds
2,2,Qaqortoq,60.7167,-46.0333,33.80,25,1,24.16,GL,2021-02-06 18:05:26,clear sky
3,3,Saldanha,-33.0117,17.9442,73.40,68,20,18.41,ZA,2021-02-06 18:05:26,few clouds
4,4,Chahal,15.7500,-89.5667,91.81,36,8,0.60,GT,2021-02-06 18:05:26,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? (in Degrees Farenheit) "))
print(f'Minimum temperature you have set is: {min_temp} degrees Farenheight')
max_temp = float(input("What is the maximum temperature you would like for your trip? (in Degrees Farenheit) "))
print(f'Maximum temperature you have set is: {max_temp} degrees Farenheight')

What is the minimum temperature you would like for your trip? (in Degrees Farenheit) 60
Minimum temperature you have set is: 60.0 degrees Farenheight
What is the maximum temperature you would like for your trip? (in Degrees Farenheit) 90
Maximum temperature you have set is: 90.0 degrees Farenheight


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                346
City                   346
Lat                    346
Lng                    346
Max Temp               346
Humidity               346
Cloudiness             346
Wind Speed             346
Country                338
Date                   346
Weather Description    346
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Sinnamary,GF,79.52,overcast clouds,5.3833,-52.9500,
3,Saldanha,ZA,73.40,few clouds,-33.0117,17.9442,
7,Antalaha,MG,77.86,broken clouds,-14.9003,50.2788,
8,Rikitea,PF,77.65,overcast clouds,-23.1203,-134.9692,
10,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,
11,Namatanai,PG,78.78,broken clouds,-3.6667,152.4333,
13,Butaritari,KI,81.01,light rain,3.0707,172.7902,
15,Hermanus,ZA,71.01,scattered clouds,-34.4187,19.2345,
16,Puerto Ayora,EC,81.00,moderate rain,-0.7393,-90.3518,
21,Chuy,UY,82.99,clear sky,-33.6971,-53.4616,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

hotel_not_found_count=0

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    time.sleep(0.2)
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_not_found_count += 1
        print("Hotel not found... skipping.")        

print(f'{hotel_not_found_count} hotels not found')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
29 hotels not found


In [9]:
hotel_df.count()

City                   338
Country                338
Max Temp               338
Weather Description    338
Lat                    338
Lng                    338
Hotel Name             338
dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [11]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
clean_hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Sinnamary,GF,79.52,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
3,Saldanha,ZA,73.40,few clouds,-33.0117,17.9442,Blue Bay Lodge
7,Antalaha,MG,77.86,broken clouds,-14.9003,50.2788,Hotel Vitasoa
8,Rikitea,PF,77.65,overcast clouds,-23.1203,-134.9692,Pension Maro'i
10,Saint-Philippe,RE,80.60,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...,...
678,Anloga,GH,86.00,few clouds,5.7947,0.8973,Pin Drop Hotel
680,Farafangana,MG,80.49,moderate rain,-22.8167,47.8333,Miarina
686,Ugoofaaru,MV,81.81,scattered clouds,5.6667,73.0000,Ungoofaaru Inn
687,Muisne,EC,84.20,light rain,0.6000,-80.0333,Quinta Samay


In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and high of {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))